# PICSURE API test notebook

Notebook aimed at testing ongoing issues with the PIC-SURE API. Two parts, 1. Environment set-up, and 2. Ongoing issues

# Environment set-up

### Installation of external dependencies

In [1]:
list_packages <- c("jsonlite", 
                   "ggplot2",
                   "plyr",
                   "dplyr",
                   "tidyr",
                   "purrr",
                   "stringr",
                   "ggrepel",
                   "devtools")

for (package in list_packages){
     if(! package %in% installed.packages()){
         install.packages(package, dependencies = TRUE, 
                         character.only = TRUE)
     }
     library(package, character.only = TRUE)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact


The following object is masked from ‘package:jsonlite’:

    flatten


Loading required package: usethis



#### Installing latest R PIC-SURE API libraries from github

In [2]:
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)


✔  checking for file ‘/private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/RtmpkDGf1o/remotes4f721108b04a/hms-dbmi-pic-sure-r-client-d62fec5/DESCRIPTION’
─  preparing ‘picsure’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘picsure_0.1.0.tar.gz’
   



✔  checking for file ‘/private/var/folders/hm/wn0bpy0j7vl2q9gqnhhccpph0000gn/T/RtmpkDGf1o/remotes4f722c663699/hms-dbmi-pic-sure-r-adapter-hpds-3279fc3/DESCRIPTION’
─  preparing ‘hpds’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘hpds_0.1.0.tar.gz’
   


### Connecting to a PIC-SURE network

Testing environment: BioData Catalyst 

In [3]:
PICSURE_network_URL <- "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [4]:
token <- scan(token_file, what = "character")

In [5]:
myconnection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [6]:
resource <- hpds::get.resource(myconnection,
                               resourceUUID = resource_id)

### Retrieving variables dictionary from HPDS Database

NB: dictionary methods work alright, it just might be useful for getting variable names

In [7]:
random_variable_name <- "\\NHLBI TOPMed: Rare Variants for Hypertension in Taiwan Chinese (THRV)\\Laboratory Measurements\\Blood and Urine Measurements\\Insulin\\"

In [8]:
random_variable_name

[1] "\\NHLBI TOPMed: Rare Variants for Hypertension in Taiwan Chinese (THRV)\\Laboratory Measurements\\Blood and Urine Measurements\\Insulin\\"

# Errors reproduction

## Issue 1: query.anyof.add() → HTTP Error 

query method `anyof` is throwing HTTP Error, although other query methods work fine (`select`, `add`, `filter`)

In [14]:
print(random_variable_name)

[1] "\\NHLBI TOPMed: Rare Variants for Hypertension in Taiwan Chinese (THRV)\\Laboratory Measurements\\Blood and Urine Measurements\\Insulin\\"


In [15]:
my_query = hpds::new.query(resource = resource)
hpds::query.anyof.add(query = my_query, 
                      keys = random_variable_name)
facts = hpds::query.run(query = my_query, result.type = "dataframe")

In [16]:
dim(facts)

[1] 1449    1

In [17]:
head(facts)

Patient.ID
<int>
425677
425678
425679
425680
425681
425682


In [18]:
hpds::query.show(my_query)

{
    "query": {
        "fields": [

        ],
        "crossCountFields": [

        ],
        "requiredFields": [

        ],
        "anyRecordOf": [
            "\\NHLBI TOPMed: Rare Variants for Hypertension in Taiwan Chinese (THRV)\\Laboratory Measurements\\Blood and Urine Measurements\\Insulin\\"
        ],
        "numericFilters": {

        },
        "categoryFilters": {

        },
        "variantInfoFilters": [

        ],
        "expectedResultType": "DATAFRAME"
    },
    "resourceUUID": "02e23f52-f354-4e8b-992c-d37c8b9ba140"
}
 


NULL

## Issue 2: Retrieving variables dictionary from HPDS Database doesn't work when no string is specified

In [ ]:
plain_variablesDict <- hpds::find.in.dictionary(resource, "")

In [ ]:
plain_variablesDict <- hpds::find.in.dictionary(resource, "NHLBI TOPMed")

In [ ]:
plain_variablesDict

### Issue 3: Retrieving the whole dictionary from HPDS database takes forever

As compared to python, check R implementation

# Issue 4: Variable names in returned DataFrame are not the same as the ones from the dictionary

# Issue 5: Error returned by query.getResultDataFrame() when using invalid token is misleading

States that variable doesn't exist in resource, instead of saying that token is invalid.

In [ ]:
hpds::query.select.add(query = my_query, 
                      keys = consent_dic[["name"]])

# Issue 6: Filter on a categorical variable cannot accept a vector as an argument, but only a list

- Actually other methods doesn't accept vectors but list, but only try a conversion and raise a warning without consequences
- Vector and list are often used indifferently in R. Just need to add a line to convert a vector to a list.  my_vector %>% list()
- And filter yet doesn't return a specific error, but is misleading instead

## Issue 7: Query shouldn't be modified in place, but rather use assignment operator

In [1]:
# BAD, current implementation
hpds::query.filter.add(query = my_query, 
                      keys = consent_dic[["name"]], 
                      phs_copdgene)
# usual R implementation 
my_query <- hpds::query.filter.add(query = my_query, 
                      keys = consent_dic[["name"]], 
                      phs_copdgene)

ERROR: Error in hpds::query.filter.add(query = my_query, keys = consent_dic[["name"]], : objet 'my_query' introuvable


## Removing
- query.delete functions, not useful
- maybe:
    - dictionary.get.entries
    - get.crosscount


# Implementing a way to specify the query using OR instead of AND

For instance, one couldwant variables that do meet one specific query.filter() conditions, OR one other specfic query.filter() conditions. But might be hard to implement though, and not 100% sure that one need that actually.